# Análise de texto de fontes desestruturadas e Web

## Exercícios da Aula 09

Este notebook servirá para relembrarmos **requests** e **BeautifulSoup**, que serão utilizadas para baixar notícias do site da **IstoÉ Dinheiro**. Em seguida, iremos extrair informações para construir um Pandas DataFrame de títulos e descrições para prática de **RegEx**.

## Importando as bibliotecas necessárias

In [ ]:
!pip install PyPDF2

In [42]:
# para nos comunicarmos com a Web
import requests
import urllib

# para extrair informações de páginas HTML
import bs4
from bs4 import BeautifulSoup

# Para criar um Data Frame
import pandas as pd

# Para expressões regulares
import re

# Para PDFs
import PyPDF2 as pp

# Recursos do sistema
import os

print('O seu notebook está na pasta:')
print(os.getcwd())

O seu notebook está na pasta:
/home/jovyan


## Definindo cabeçalho User-Agent

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
}

## Definindo em qual página buscar

Vamos definir qual seção iremos utilizar para baixar as notícias.

In [ ]:
secao = 'economia'
url = f'https://www.istoedinheiro.com.br/categoria/{secao}/'

## Utilizando *requests* para baixar a página de notícias

Com o uso da biblioteca **requests**, podemos obter o **HTML** da página da IstoÉ.

In [ ]:
resposta = requests.get(url = url, headers=headers)

resposta.encoding = 'utf-8'

resposta.text

## Extraindo informações relevantes com *BeautifulSoup*

Perceba que o HTML inclui uma grande quantidade de tags, o que dificulta identificar informações relevantes de forma direta. Com o auxílio da biblioteca **BeautifulSoup** podemos extrair facilmente as informações que desejamos.

In [ ]:
soup = BeautifulSoup(resposta.text, 'html.parser')

Agora, vamos obter uma lista com todos os trechos HTML que contém uma notícia. Para identificar as tags corretas, é preciso ir até a página Web que desejamos extrair informações, dar botão direito e ir em **inspecionar elemento**, navegando pelo HTML até identificar as tags necessárias

Ex: https://www.istoedinheiro.com.br/categoria/economia/politica/

In [ ]:
lista_tag_noticia = soup.find_all('article', class_='thumb')

Agora, podemos passar por cada uma das chamadas de notícias, extraindo informações de interesse, como o título, descrição e data.

In [ ]:
lista_titulo = []
lista_desc = []
lista_data = []

for i in range(0, len(lista_tag_noticia)):
    
    tag_noticia = lista_tag_noticia[i]


    titulo = tag_noticia.find('h3').text
    titulo = titulo.replace('\n', '') #limpa os ENTERS a esquerda e direita
    lista_titulo.append(titulo)

    descricao = tag_noticia.find('p').text
    lista_desc.append(descricao)
    
    data_hora = tag_noticia.find('time', class_='c-data').text
    lista_data.append(data_hora)

## Criando um DataFrame

As informações que consideramos foram extraídas na repetição **for** e armazenadas em listas. Podemos utilizar estas listas para construir um Pandas DataFrame:

In [ ]:
df = pd.DataFrame({'Secao': secao,
                   'Titulo': lista_titulo,
                   'Descrição': lista_desc,
                   'Data': lista_data
                  })
df

## Fazer o scraping de várias páginas

Ao navegar pelo site, percebemos que a **URL** das páginas segue o seguinte padrão: https://www.istoedinheiro.com.br/categoria/economia/page/2/

Vamos construir algumas funções auxiliares e fazer a extração dos dados das notícias de várias páginas:

In [ ]:
def get_html_of_page(secao = 'economia', page_number=1):
    url = f'https://www.istoedinheiro.com.br/categoria/{secao}/page/{page_number}'
    headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
    resposta = requests.get(url = url, headers=headers)
    resposta.encoding = 'utf-8'
    return resposta.text

def get_data_of_page(secao = 'economia', page_number=1):
    html = get_html_of_page(secao=secao, page_number=page_number) 
    soup = BeautifulSoup(html, 'html.parser')
    lista_tag_noticia = soup.find_all('article', class_='thumb')

    lista_titulo = []
    lista_desc = []
    lista_data = []

    for i in range(0, len(lista_tag_noticia)):

        tag_noticia = lista_tag_noticia[i]

        titulo = tag_noticia.find('h3').text
        titulo = titulo.replace('\n', '') #limpa os ENTERS a esquerda e direita
        lista_titulo.append(titulo)

        descricao = tag_noticia.find('p').text
        lista_desc.append(descricao)

        data_hora = tag_noticia.find('time', class_='c-data').text
        lista_data.append(data_hora)

    return lista_titulo, lista_desc, lista_data

def get_dataframe_of_page(secao = 'economia', page_number=1):
    lista_titulo, lista_desc, lista_data = get_data_of_page(secao=secao, page_number=page_number)
    df = pd.DataFrame({'Secao': secao,
                       'Titulo': lista_titulo,
                       'Descrição': lista_desc,
                       'Data': lista_data
                      })
    return df

def get_news(secoes, n_pages=5):
    dfs = []
    for secao in secoes:
        for i in range(1, n_pages+1):
            dfs.append(get_dataframe_of_page(secao=secao, page_number=i))
    return pd.concat(dfs, axis=0)        

Vamos extrair dados de algumas seções, sendo das páginas 1..5 para cada seção.

In [ ]:
df = get_news(['economia', 'negocios', 'mercado-digital'], n_pages=5)

E visualizar algumas notícias

In [ ]:
df.sample(10)

### Salvando o DataFrame em CSV

É interessante armazenar o Dataframe em CSV para que ele possa ser análisado em algum momento posterior. Você poderia, por exemplo, extrair as notícias todos os dias de uma semana e analisar somente após ter todos estes dados.

In [ ]:
df.to_csv('noticias_200423.csv', index=False)

In [ ]:
df = pd.read_csv('notebooks/lessons/09/noticias_200423.csv')
df

## Relembrando - Extração de textos de PDFs

Vamos relembrar o que vimos na aula de extração de textos de PDFs e juntar com Expressões Regulares para procurarmos por padrões interessantes.

Primeiro, vamos fazer o download de uma página qualquer do diário oficial.

In [ ]:
pdf_url="http://diariooficial.imprensaoficial.com.br/doflash/prototipo/2023/Abril/19/exec1/pdf/pg_0001.pdf"

response = urllib.request.urlopen(pdf_url)

Vamos salvar a resposta obtida em um arquivo PDF chamado `'pg_0001.pdf'`

In [ ]:
with open('pg_0001.pdf', 'wb') as arq:
    arq.write(response.read())


Então podemos utilizar a biblioteca `PyPDF2` para extrair os textos. Aqui, pense que nosso objetivo será apenas identificar **CPF**, **CNPJ**, datas, projetos de leis, etc. mencionados, não importanto tanto que o texto esteja em ordem.

In [ ]:
pp_reader = pp.PdfReader(open('pg_0001.pdf', 'rb'))

texto = pp_reader.pages[0].extract_text()

texto

### Capturando CPFs

Vamos procurar por todos os CPFs mencionados nesta página:

In [ ]:
re.findall(r'\d{3}\.\d{3}\.\d{3}-\d{2}', texto)

E procurar por todas as datas

In [ ]:
re.findall(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', texto)

**Pergunta**: Qual a utilidade do `\b` no regex acima?

<div class="alert alert-success">

O `\b` vem do inglês "Word Boundary" que significa "limite da palavra", ou seja, é esperado que naquela posição do padrão se esteja no limite da palavra.
Neste caso está servindo de garantia que se está procurando por uma data sem contaminação. Por exemplo "125812/09/200801".

</div>

# Exercícios

Considere a base de notícias recém extraída para os exercícios.

**Exercício 1)** Utilize RegEx para criar uma nova coluna no DataFrame. Esta nova coluna deve indicar se o título da notícia tem faz ou não menção ao **governo**.

Aqui, você vai ter que pensar em um RegEx que busque por termos que façam sentido e generalizem a noção de **"governo"**.

In [ ]:
pattern_government = r"\bpresidente|govern(?:ador|adora|o)|prefeut(?:o|a|ura)|minist(?:r[oa]s?|érios?)|embaixad(?:a|or|ora)|parlament(?:o|ar)|aliad(?:[oa]s?)|brasil|acordo|bc\b"
matches = df["Titulo"].str.contains(pattern_government, flags=re.IGNORECASE)

df["governo_titulo"] = matches.astype("category").cat.rename_categories({
    True: "sim",
    False: "não",
})

df

**Exercício 2)** Repita o exercício anterior, buscando por termos que façam menção à **bolsas de valores**.

In [ ]:
pattern_stock_exchanges = r"\bbolsas?|ibovespa|sp500|nasdaq|alta|baixa|acionistas?|fundo|crescimento\b"
matches = df["Titulo"].str.contains(pattern_stock_exchanges, re.IGNORECASE)

df["bolsas_titulo"] = matches.astype("category").cat.rename_categories({
    True: "sim",
    False: "não",
})

df

**Exercício 3)** Repita o exercício anterior, buscando por termos que façam menção à **bolsas de valores** ou ao **governo** na **descrição** das notícias.

In [ ]:
matches_government = df["Descrição"].str.contains(pattern_government, re.IGNORECASE)
matches_stock_exchanges = df["Descrição"].str.contains(pattern_stock_exchanges, re.IGNORECASE)
matches = matches_government | matches_stock_exchanges

df["governo_bolsas_descricao"] = matches.astype("category").cat.rename_categories({
    True: "sim",
    False: "não",
})

df

**Exercício 4)** Utilizando as variáveis novas criadas com RegEx:


**a)** Conte quantas notícias fazem menção ao Governo. Conte para o título e também descrição.

In [ ]:
count_governo = df["governo_titulo"].value_counts() + \
                df["governo_bolsas_descricao"].value_counts()

f"{count_governo['sim']} notícias fazem menção ao Governo."

**b)** Gere um gráfico de barras das frequências absolutas (contagem)

In [ ]:
df.iloc[:, 4:7].value_counts(["governo_titulo"])

**c)** E se quisermos a frequência relativa (porcentagem)

**Exercício 5)** Calcule a frequência de menções à Pandemia por seção (política, economia, finanças). Obs: Considere a descrição da notícia. Gere um gráfico de barras dos resultados.

In [ ]:
pattern_pandemia = r"\bpandemia|covid|coronavirus\b"

df.loc[:, ("Descrição", "Secao")].groupby("Secao").sum()["Descrição"].str.count(pattern_pandemia, re.IGNORECASE)

**Exercício 6)** Crie um código python que consiga fazer o download de várias páginas do diário oficial.

Salve em arquivos nomeados no padrão `'pg_0001.pdf'`, `'pg_0002.pdf'`, ... , `'pg_000n.pdf'`

In [ ]:
num_paginas_diario_oficial = 5

for index in range(1, num_paginas_diario_oficial+1):
    filename = f"pg_{index:04}.pdf"
    pdf_url=f"http://diariooficial.imprensaoficial.com.br/doflash/prototipo/2023/Abril/19/exec1/pdf/" + filename
    response = urllib.request.urlopen(pdf_url)

    with open(f'notebooks/lessons/09/diario_oficial/{filename}', 'wb') as pdf_file:
        pdf_file.write(response.read())


**Exercício 7)** Crie um código python que leia `n` arquivos **PDF**s de uma pasta e extraia seus textos utilizando a biblioteca vista no exemplo da aula.

Aqui, cada **PDF** é uma página do diário oficial. Suponha que os arquivos estão nomeados no padrão `'pg_0001.pdf'`, `'pg_0002.pdf'`, ... , `'pg_000n.pdf'`.

Retorne uma lista onde cada item da lista é uma string contendo o texto da página em questão. Exemplo:

```python
[
    'texto da página 01 do diário`,
    'texto da página 02 do diário`,
    'texto da página 03 do diário`,
    'texto da página 04 do diário`,
]
```

In [ ]:
directory = os.listdir("notebooks/lessons/09/diario_oficial/")
pdf_filenames = [filename for filename in directory if filename.endswith(".pdf")]

pdf_texts = []

for filename in pdf_filenames:
    file_path = f"notebooks/lessons/09/diario_oficial/{filename}"
    with open(file_path, 'rb') as pdf_file:
        pdf = pp.PdfReader(pdf_file)

        pdf_texts.append(pdf.pages[0].extract_text() )

pdf_texts

**Exercício 8)** Crie um código python que recebe a lista do exercício anterior. Exemplo:

```python
[
    'texto da página 01 do diário`,
    'texto da página 02 do diário`,
    'texto da página 03 do diário`,
    'texto da página 04 do diário`,
]
```

Você deve procurar todos os **CPF**s ou **CNPJ**s contidos no diário. Indique a página onde o mesmo foi encontrado. Exemplo de resposta:
```python
[
    ['123.456.789-00', 0],
    ['87.340.538/0001-23', 0]
    ['555.666.777-00', 1],
    ['30.375.316/0001-29', 3],
    ['30.375.316/0001-29', 3],
]
```

In [ ]:
cpfs = [re.findall(r'\d{3}\.\d{3}\.\d{3}-\d{2}', text) for text in pdf_texts]
cpfs

**Exercício 9)** Um empresário deseja saber quais páginas do diário oficial fazem menção a determinado assunto.

Faça um programa em python que recebe os textos das páginas do diário oficial:


```python
[
    'texto da página 01 do diário`,
    'texto da página 02 do diário`,
    'texto da página 03 do diário`,
    'texto da página 04 do diário`,
]
```

Crie um padrão de expressão regular para busca por termos/palavras (simule um cenário) e retorne uma lista das páginas que fazem menção ao padrão. Exemplo de resposta:


```python
[0, 2, 3]
```

In [ ]:
pattern = r"\bsão paulo\b"
re_pattern = re.compile(pattern, re.IGNORECASE)

matches = [re_pattern.findall(text) for text in pdf_texts]
pages = [index for index, page_matches in enumerate(matches) if len(page_matches) > 0]

pages